In [ ]:
%%shell
gsutil cp gs://brokenegg/pretrained/model_base_20200623.tar.bz2 .
tar xfj model_base_20200623.tar.bz2

Copying gs://brokenegg/pretrained/model_base_20200623.tar.bz2...
| [1 files][819.1 MiB/819.1 MiB]   53.6 MiB/s                                   
Operation completed over 1 objects/819.1 MiB.                                    


In [1]:
from brokenegg_transformer import transformer
from brokenegg_transformer import model_params
from brokenegg_transformer.utils import tokenizer
import tensorflow.compat.v1 as tf
import numpy as np

In [2]:
# Creating translation model
params = model_params.BASE_PARAMS.copy()
params["dtype"] = tf.float32
with tf.name_scope("model"):
    model = transformer.create_model(params, is_train=False, has_initial_ids=True)
init_weight_path = tf.train.latest_checkpoint("./model_base_20200623")
print('Restoring from %s' % init_weight_path)
checkpoint = tf.train.Checkpoint(model=model)
checkpoint.restore(init_weight_path)

Restoring from ./model_base_20200623/ctl_step_760000.ckpt-38


In [3]:
# Creating tokenizer
path = './model_base_20200623/brokenegg.en-es-ja.spm64k.model'
subtokenizer = tokenizer.Subtokenizer(path)

INFO:tensorflow:Initializing Subtokenizer from file ./model_base_20200623/brokenegg.en-es-ja.spm64k.model.


In [5]:
#text = 'US is located in the west.'
text = 'Hello! I am going to school.'
#text = 'Note: this transformer folder is subject to be integrated into official/nlp folder.'
text = 'They include President Trump\'s former lawyer Michael Cohen who has served a prison sentence for lying to Congress and campaign finance fraud.'
#text = '元気ですか？'
#text = 'Are you sure?'
#text = '¿Estás seguro?'
#text = 'どこ行こうか？'
text = '愛している'

# task_id is one of 1, 64000, 64001, 64002
# 1: Conversation
# 64000: Translate from Spanish/Japanese to English
# 64001: Translate from English/Japanese to English
# 64002: Translate from English/Spanishe to Japanese

task_id = 1
encoded = subtokenizer.encode(text, add_eos=True)
output, score = model.predict([np.array([encoded], dtype=np.int64), np.array([task_id], dtype=np.int32)])
print(output)
encoded_output = []
for _id in output[0]:
    _id = int(_id)
    if _id == tokenizer.EOS_ID:
        break
    encoded_output.append(_id)
decoded_output = subtokenizer.decode(encoded_output)
print(decoded_output)

[[  6  21 450  21 450  21 450  21 450  21 450  21 450  21 450  21 450  21
  450  21 450  21 450  21 450  21 450   2   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]]
はいはいはいはいはいはいはいはいはいはいはいはいはい
